In [1]:
# Package imports
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import pandas as pd
import numpy as np
from decimal import Decimal
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from mnist import MNIST
import datetime


In [2]:
# set up training rate alpha
alpha = 0.5

# Use pandas to read the CSV file as a dataframe
df = pd.read_csv("moons400.csv")
# The y values are those labelled 'Class': extract their values
y = df['Class'].values

# The x values are all other columns
#del df['Class']    # drop the 'Class' column from the dataframe
X = df.as_matrix() # convert the remaining columns to a numpy array
# Some examples of working with the data, to look at rows/columns
print ("len(X):", len(X))            # outer array: one per sample
print ("len(X[0]):", len(X[0]))      # each inner array is the attributes of one sample
print ("len(X[:,0]):", len(X[:,0]))  # select column 0 from array

# np.shape returns all dimensions of the array
(nsamples, nattribs) = np.shape(X)
print ("X: nsamples =", nsamples, ", nattribs =", nattribs)

# using sklearn.model_selection.train_test_split to split up data into train and test sets split 70/30
train_X, test_X, train_y, test_y = train_test_split(df, y, test_size=0.30)

print(train_X.shape, train_y.shape)
print(test_X.shape, test_y.shape)

print("length train X", len(train_X))
print("length test X", len(test_X))
print("length train y", len(train_y))
print("length test y", len(test_y))

# The x train and test values are all other columns
#print(train_X)
del train_X['Class']    # drop the 'Class' column from the Train and test dataframe
del test_X['Class']
#print(test_X)

print("train: ", train_X.shape, train_y.shape)
print("test: ", test_X.shape, test_y.shape)

len(X): 400
len(X[0]): 3
len(X[:,0]): 400
X: nsamples = 400 , nattribs = 3
(280, 3) (280,)
(120, 3) (120,)
length train X 280
length test X 120
length train y 280
length test y 120
train:  (280, 2) (280,)
test:  (120, 2) (120,)


In [4]:
#datasubset_x = X[0:2]
datasubset_x = train_X[0:2]
datasubset_x = train_X
(dsamples, dattribs) = np.shape(datasubset_x)
print("datasubset: dsamples =", dsamples, ", dattribs =", dattribs)

# print(datasubset_x)

datasubset: dsamples = 280 , dattribs = 2


In [5]:
# rescale the inputs using normalization 
inputs = preprocessing.normalize(datasubset_x)
# inputs = datasubset_x
# inputs[0,0] = 0.05
# inputs[0,1] = 0.1
# print(inputs)
inputs = np.array(inputs, dtype=np.float64)

In [6]:
# target = y[0:20]
target = train_y[0:2]
target = train_y
# target = 0.01
#print(target)
target = np.array(target, dtype=np.float64)

In [7]:
# set up training rate alpha
alpha = 0.1

# initialise the weights for the network based on the input layers, the number of hidden layers, the number of output layers
def initialise_input_weights(n_inputs, n_hidden_inputs):
 hidden_layer_weights = list()
 for i in range(n_hidden_inputs):
  #for j in range(n_inputs):
   weight = np.random.randn(n_inputs)*np.sqrt(1/(n_inputs)**(n_hidden_inputs-1)) 
    #reference for above https://www.coursera.org/learn/deep-neural-network/lecture/RwqYe/weight-initialization-for-deep-networks
   hidden_layer_weights.append(weight)
 
 input_weights = np.array([hidden_layer_weights])
 input_weights = np.reshape(input_weights, (n_inputs, n_hidden_inputs))
 return input_weights; 

def initialise_output_weights(n_hidden_inputs,n_outputs):
 output_layer_weights = list()
 for i in range(n_outputs):
  for j in range(n_hidden_inputs):
   weight = np.random.random(1)[0]
   output_layer_weights.append(weight) 
  
 if n_outputs == 1:
  output_weights = np.array([output_layer_weights])
 elif n_outputs > 1:
  output_weights = np.array([output_layer_weights])  
  output_weights = np.reshape(output_weights, (n_hidden_inputs,n_outputs))
 
 return output_weights;
    
#initialise the bias for the network based on the number of hidden layers and the output layer bias
def initialise_bias(n_hidden_layer):
 hidden_layer_bias = list()    
 for i in range(n_hidden_layer):
  bias = np.random.random(1)[0]
  hidden_layer_bias.append(bias)
 
 output_layer_bias = [np.random.random(1)[0]]
 network_bias = [[hidden_layer_bias],[output_layer_bias]]
 return network_bias;

In [8]:
# FORWARD PROPAGATION USING FUNCTIONS
# do the above using a function
# convert calculations to function for hidden layer
def get_sigmoid_output(weights, input, bias):
    hidden_layer = input * weights 
    hidden_layer = np.sum(hidden_layer, axis=1) + bias
    hidden_layer = 1/(1+np.exp(-hidden_layer))
    return hidden_layer;

def get_output(sigmoid_out, weights, bias):
    output = (sigmoid_out*weights)
    output = 1/(1+np.exp(-(np.sum(output)+bias)))
    return output;

# get the error 
# error_o = (0.5*(target - output)**2)
# print(error_o)

#convert to a function
# I think this needs to change to accept a numpy array as well
# we should end up with a data frame we can plot against the number of executions
def get_error(t, o):
    error_o = (0.5*(t - o)**2)
    return error_o;


In [9]:
# FUNCTIONS FOR BACK PROPAGATION OF OUTPUT
# calculate derivative of error at output layer

# deriv_wrt_out = -(target - output)
# change above to function
def deltaErr_wrt_out(targ, outp):
    result = -(targ - outp)
    return result;  

# calculate the derivation of the error output wrt the net
# derivout_wrt_net = output*(1-output)
# change above to function
def deltaOut_wrt_net(outp):
    result = outp*(1-outp)
    return result;   

# change above to function
# calculate derivative of error wrt to output layer weight OLW_Deriv
def deltaErr_ow(deriv_wrt_out, derivout_wrt_net, activation):
    OLW_Deriv = deriv_wrt_out*derivout_wrt_net*activation
   # print(OLW_Deriv)
    return OLW_Deriv;

In [10]:


# FUNCTIONS FOR BACK PROPAGATION OF HIDDEN LAYER
# calculate derivative of error at hidden layer

# deriv_out_wrt_hL =  Weights2 * deriv_wrt_out *derivout_wrt_net
# print (deriv_out_wrt_hL)
# convert above to function
def deltaOut_hL(deriv_wrt_out, derivout_wrt_net, weights):
    deriv_out_wrt_hL = deriv_wrt_out *derivout_wrt_net *  weights
    return deriv_out_wrt_hL;

# deriv_out_wrt_nethL = activation*(1-activation)
# convert above to function
def deltaOut_netHL(activation):
    activation = activation*(1-activation)
    return activation;

# deriv_wrt_wi = deriv_out_wrt_hL*deriv_out_wrt_nethL*Weights1
# convert above to function
def deltaErr_wi(deriv_out_wrt_hL, deriv_out_wrt_nethL, weights):
    deriv_wrt_wi = deriv_out_wrt_hL*deriv_out_wrt_nethL*weights
    return deriv_wrt_wi;

# convert the above to a function
def calc_adjusted_weights(W, deriv):
    W = W - (alpha*deriv)
    return W;


In [12]:
# set up weights and biases 
(train_samples, train_shape) = np.shape(inputs)
print ("train: samples =", train_samples, ", attribs =", train_shape)

# Test weights and bias initialisation based on network inputs, hidden layers, and outputs
# first line of moons.csv
#input_X =[[2.07106946, 0.41152931]]

#(nsamples, nattribs) = np.shape(input_X)
n_inputs = train_shape

n_hidden_layer = 1
n_hidden_inputs = 2
n_outputs = 1

# normalise training data
# rescale the inputs using normalization 
# mnist_train = preprocessing.normalize(filtered_images)
#print("normalised inputs",inputs)

Weights1 = np.array(initialise_input_weights(n_inputs, n_hidden_inputs), dtype=np.float64)
Weights2 = np.array(initialise_output_weights(n_hidden_inputs, n_outputs), dtype=np.float64)
#network_weights = initialise_weights(n_inputs, n_hidden_inputs, n_outputs)
bias = np.array(initialise_bias(n_hidden_layer), dtype=np.float64)

print("Length of inputs",len(inputs))
#print("actual y",input_y)
print("input Weights", len(Weights1))
print(Weights1)
print("output Weights", len(Weights2))
print("Network Bias", len(bias))

train: samples = 280 , attribs = 2
Length of inputs 280
input Weights 2
[[ 0.28866915 -0.60956944]
 [ 0.12669093  0.17858478]]
output Weights 1
Network Bias 2


In [35]:
print("Length of inputs",len(inputs))
#print("actual y",input_y)
print("input Weights", len(Weights1))
print("output Weights", len(Weights2))
print("Network Bias", len(bias))

print(Weights1)
print (Weights2)
print(bias)
#print(deltaErrOut_hL)

Length of inputs 120
input Weights 2
output Weights 1
Network Bias 2
[[ 10.36698256  -1.7652734 ]
 [  4.54985476   0.51716989]]
[[ 7.20885252 -1.63263479]]
[[[-9.99336772]]

 [[-1.68657535]]]


In [15]:
# use a loop to iterate through the dataset
# and present records 1 by 1
# for i in np.nditer(inputs, flags=['external_loop'], order='C'):
#for i in range(len(inputs)):
#    print (i)
#    for t in range((target[i])):## this part is not working because it loops through the whole target array each time
#        print(t)  ## not what we want
alpha = 0.01
#TRAIN MODEL
print(datetime.datetime.now())
threshold=1e-5
maxrounds=3000
iter=0
error = 99.0
while abs(error) > threshold:
    for i in range(len(inputs)):
        
        # START OF FORWARD PROPAGATION FUNCTION CALLS
        # test get_sigmoid_output 
        hidden_layer = get_sigmoid_output(Weights1, inputs[i], bias[0])
        #print ("hidden_layer result:", hidden_layer)

        # test get_output
        output = get_output(hidden_layer, Weights2, bias[1])
        #print ("output:", output)

        # call error function
        error = get_error(target[i], output)
       # print("error:", error)

        # END OF FORWARD PROPAGATION FUNCTION CALLS

        # START OF BACK PROPAGATION FOR OUTPUT LAYER
        #test function deltaErr_wrt_out
        deltaErr_out = deltaErr_wrt_out(target[i], output)
        # print(deltaErr_out)

        #test function
        deltaOut_net = deltaOut_wrt_net(output)
        # print(deltaOut_net)

        deltaErrtot_ow = deltaErr_ow(deltaErr_out, deltaOut_net, hidden_layer)
        # print(deltaErrtot_ow)

        # END OF BACK PROPAGATION FOR OUTPUT LAYER

        # START BACK PROPAGATION OF HIDDEN LAYER
        # calculate derivative of error at hidden layer

        # test function deltaOut_hL
        deltaErrOut_hL = deltaOut_hL(deltaErr_out, deltaOut_net, Weights2)
        # print(deltaErrOut_hL)

        #test function 
        deltaErrOut_netHL = deltaOut_netHL(hidden_layer)
        # print(deltaErrOut_netHL)

        deltaErrH_wi = deltaErr_wi(deltaErrOut_hL, deltaErrOut_netHL, Weights1)
        # print(deltaErrH_wi)

        # same calculation as above but with Weights matrix 
        # calculate adjusted weights using function
        Weights2 = calc_adjusted_weights(Weights2, deltaErrtot_ow)
        Weights1 = calc_adjusted_weights(Weights1, deltaErrH_wi)
        bias[1] = calc_adjusted_weights(bias[1], deltaErr_out)
        bias[0] = calc_adjusted_weights(bias[0], 0.5*np.sum(deltaErrOut_hL, dtype=np.float64))

        # END OF BACK PROPAGATION OF HIDDEN LAYER

    iter=iter+1
    
    if (iter > maxrounds):
        break

print ("\nFinished after ", iter, " error =", error, "target=", target[i], ", output=", output, "input final=", inputs[i])    
print(datetime.datetime.now())
print ("Weights1 adjusted:", Weights1)
print ("Weights2 adjusted:", Weights2)

2018-02-20 20:56:01.941430

Finished after  3001  error = [[ 0.01223564]] target= 0.0 , output= [[ 0.15643296]] input final= [-0.99838555  0.05680041]
2018-02-20 20:56:41.344958
Weights1 adjusted: [[ 10.36698256  -1.7652734 ]
 [  4.54985476   0.51716989]]
Weights2 adjusted: [[ 7.20885252 -1.63263479]]


In [17]:
# reference https://pypi.python.org/pypi/python-mnist
# https://github.com/sorki/python-mnist/blob/master/mnist/loader.py
# TRAIN MNIST DATA

mndata = MNIST('./mnist')
images, labels = mndata.load_training()

processed_images = mndata.process_images_to_numpy(images)
# print(processed_images[0:2])

target = np.array(labels)
# print(target[0:2])

In [19]:
# set up 2 lists to filter the MNIST data into
# keeping only 0 and 6 to classify
# TRAIN MNIST DATA

filtered_labels = []
filtered_images = []
for i in range(len(target)):
    if target[i]==0 or target[i]==6:
        filtered_labels.append(target[i])
        filtered_images.append(processed_images[i])
        
# remap the value 6 to 1, so classification is binary

#print(filtered_labels[0:10])

for i in range(len(filtered_labels)):
    if filtered_labels[i]==6:
        filtered_labels[i]=1

#print(filtered_labels[0:10])

[0, 6, 6, 0, 6, 0, 6, 0, 6, 0]
[0, 1, 1, 0, 1, 0, 1, 0, 1, 0]


In [20]:
# convert the lists to arrays
# TRAIN MNIST DATA
filtered_labels = np.array(filtered_labels)

filtered_images = np.array(filtered_images)

In [23]:
# MNIST TEST DATA
test_images, test_labels = mndata.load_testing()

test_images = mndata.process_images_to_numpy(test_images)
#print(processed_images[0:2])

test_target = np.array(test_labels)

# set up 2 lists to filter the MNIST data into
# keeping only 0 and 6 to classify
# TEST MNIST DATA

filtered_test_labels = []
filtered_test_images = []
for i in range(len(target)):
    if target[i]==0 or target[i]==6:
        filtered_test_labels.append(target[i])
        filtered_test_images.append(processed_images[i])
        
# remap the value 6 to 1, so classification is binary

#print(filtered_labels[0:10])

for i in range(len(filtered_test_labels)):
    if filtered_test_labels[i]==6:
        filtered_test_labels[i]=1
        
# convert the lists to arrays
# TEST MNIST DATA
filtered_test_labels = np.array(filtered_test_labels)

filtered_test_images = np.array(filtered_test_images)        

[0 1 1 0 1 0 1 0 1 0]


In [30]:
print(filtered_test_labels[0:10])


[0, 1, 1, 0, 1, 0, 1, 0, 1, 0]


In [24]:
# set up weights and biases for MNIST data, get nattribs value
# TRAIN MNIST DATA

(train_samples, train_shape) = np.shape(filtered_images)
print ("train: samples =", train_samples, ", attribs =", train_shape)

# Test weights and bias initialisation based on network inputs, hidden layers, and outputs
# first line of moons.csv
#input_X =[[2.07106946, 0.41152931]]

#(nsamples, nattribs) = np.shape(input_X)
n_inputs = train_shape
n_hidden_layer = 1
n_hidden_inputs = 2
n_outputs = 1

# normalise training data
# rescale the inputs using normalization 
mnist_train = preprocessing.normalize(filtered_images)
#print("normalised inputs",inputs)

input_weights = np.array(initialise_input_weights(n_inputs, n_hidden_inputs))
output_weights = np.array(initialise_output_weights(n_hidden_inputs, n_outputs))
#network_weights = initialise_weights(n_inputs, n_hidden_inputs, n_outputs)
network_bias = np.array(initialise_bias(n_hidden_layer))

print("Length of inputs",len(filtered_images))
#print("actual y",input_y)
print("input Weights", len(input_weights))
print("output Weights", len(output_weights))
print("Network Bias", len(network_bias))

train: samples = 11841 , attribs = 784
Length of inputs 11841
input Weights 784
output Weights 1
Network Bias 2


In [31]:
## TEST MODEL
inputs = preprocessing.normalize(test_X)
target = test_y
target = np.array(target)
print(len(inputs))
print(len(target))
#tol=1e-7
#maxrounds=1000
#iter=0
error = 99.0
#while abs(error) > tol:
error_res = []
result = []
#result = np.array(result)

for i in range(len(inputs)):
    print ("Index: ",i, " Input: ", inputs[i])
    print ("Index: ",i," Target value: ", target[i])
        
    ### START OF FORWARD PROPAGATION FUNCTION CALLS
    # test get_sigmoid_output 
    hidden_layer = get_sigmoid_output(Weights1, inputs[i], network_bias[0])
    ##print ("hidden_layer result:", hidden_layer)
        
    # test get_output
    output = get_output(hidden_layer, Weights2, network_bias[1])
    #  print ("output:", output)
    result.append(output)
        
    # call error function
    error = get_error(target[i], output)
    error_res.append(error)
    #print("error:", error)
        
    # END OF FORWARD PROPAGATION FUNCTION CALLS
result = np.array(result)  
error_res = np.array(error_res)
target = np.array(target)
#print("Target:", target)        
#print("Results: ", result)
#print("Error:", error_res)

120
120
Index:  0  Input:  [ 0.48689889  0.87345834]
Index:  0  Target value:  0
Index:  1  Input:  [ 0.99547416 -0.09503259]
Index:  1  Target value:  1
Index:  2  Input:  [ 0.66817981 -0.74399982]
Index:  2  Target value:  1
Index:  3  Input:  [ 0.99654419  0.08306432]
Index:  3  Target value:  0
Index:  4  Input:  [-0.72962975  0.6838424 ]
Index:  4  Target value:  0
Index:  5  Input:  [-0.17434003  0.98468551]
Index:  5  Target value:  1
Index:  6  Input:  [-0.40422617  0.91465907]
Index:  6  Target value:  0
Index:  7  Input:  [-0.03451603  0.99940414]
Index:  7  Target value:  1
Index:  8  Input:  [ 0.99652197  0.08333043]
Index:  8  Target value:  1
Index:  9  Input:  [-0.4812387   0.87658959]
Index:  9  Target value:  0
Index:  10  Input:  [-0.17222527  0.98505759]
Index:  10  Target value:  0
Index:  11  Input:  [ 0.19469947  0.98086295]
Index:  11  Target value:  0
Index:  12  Input:  [ 0.48120095  0.87661032]
Index:  12  Target value:  0
Index:  13  Input:  [-0.44670859  0.8

In [34]:
# calculate the accuracy of the NN
tol=1e-7
print(len(target))
print(len(result))
print(tol)
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(len(target)):
        
        if (target[i] == 1 and target[i]-result[i] <= tol):
            TP = TP + 1
            print("True Positive: ", target[i], " - ", result[i])

        if (target[i] == 0 and target[i]-result[i] <= tol):
            TN = TN + 1
            print("True Negative: ",target[i], " - ", result[i])
            
        if (target[i] == 1 and target[i]-result[i] > tol):
            FP = FP + 1
            print("False Positive: ",target[i], " - ", result[i])
            
        if (target[i] == 0 and target[i]-result[i] > tol):
            FN = FN + 1
            print("False Negative: ",target[i], " - ", result[i])
        
print("True Positive: ", TP)
print("True Negative: ", TN)
print("False Positive: ", FP)
print("False Negative: ", FN)


120
120
1e-07
True Negative:  0  -  [[ 0.99848159]]
False Positive:  1  -  [[ 0.99856238]]
False Positive:  1  -  [[ 0.99862]]
True Negative:  0  -  [[ 0.99856124]]
True Negative:  0  -  [[ 0.6886039]]
False Positive:  1  -  [[ 0.59181311]]
True Negative:  0  -  [[ 0.60250123]]
False Positive:  1  -  [[ 0.77430094]]
False Positive:  1  -  [[ 0.99856124]]
True Negative:  0  -  [[ 0.62771788]]
True Negative:  0  -  [[ 0.59302581]]
True Negative:  0  -  [[ 0.99246221]]
True Negative:  0  -  [[ 0.99847377]]
False Positive:  1  -  [[ 0.61642911]]
True Negative:  0  -  [[ 0.71331424]]
False Positive:  1  -  [[ 0.99856125]]
True Negative:  0  -  [[ 0.7100974]]
True Negative:  0  -  [[ 0.99779114]]
False Positive:  1  -  [[ 0.99866076]]
True Negative:  0  -  [[ 0.99852397]]
True Negative:  0  -  [[ 0.99856832]]
False Positive:  1  -  [[ 0.99856157]]
True Negative:  0  -  [[ 0.99052377]]
False Positive:  1  -  [[ 0.99856121]]
False Positive:  1  -  [[ 0.99856161]]
False Positive:  1  -  [[ 0.99